### web crawling
- 웹 서비스의 구조: 서버-클라이언트
- 크롤링 방법 세가지
    - requests: json 동적 페이지
        - 네이버 주식 사이트에서 주가 데이터 크롤링
            - 시각화(정규화), 상관계수
    - requests: html 정적 페이지
    - selenium: webdriver
- 크롤링 절차
    - 웹서비스 분석: URL
    - 요청, 응답: str
    - str 데이터 파싱(dict, bs obj(css-selector)) 후 데이터 프레임으로 생성

###  직방
- 동이름을 입력하면 아파트 매물을 데이터 프레임 만들기
- robots.txt

### 1. 웹서비스 분석:  URL 찾기, 크롤링 전략 세우기
- + URL의 특수문자는 인코딩된 데이터 그래서 디코딩해서 값을 줘야함
- + geohash: 위도, 경도 값을 문자열 형태로 바꿔주는 패키지
- 동이름 입력 -> lat, lng (위도, 경도)
- lat, lng -> geohash: geohash 패키지 설치
- geohash -> item ids
- item ids -> item datas(데이터 프레임)

In [32]:
# 동이름 -> 위도경도
addr = '성수동'
# url = 'https://apis.zigbang.com/v2/search?q=%EC%84%B1%EC%88%98%EB%8F%99&serviceType=%EC%95%84%ED%8C%8C%ED%8A%B8'.format(addr)
    # 디코딩 필요한 URL
url = 'https://apis.zigbang.com/v2/search?q={}&serviceType=아파트'.format(addr)
response = requests.get(url)
data = response.json()['items'][0]
lat, lng = data['lat'], data['lng']
data, lat, lng

({'id': 4094,
  'type': 'address',
  'name': '성수동1가',
  'hint': '',
  'description': '서울시 성동구 성수동1가',
  'lat': 37.541725158691406,
  'lng': 127.04335021972656,
  'zoom': 5,
  'polygon': [],
  '_score': None,
  '_source': {'name_length': 5,
   'local1': '서울시',
   'local2': '성동구',
   'local3': '성수동1가',
   'web_level': 15,
   'web_lat': 37.541725158691406,
   'web_lng': 127.04335021972656,
   'app_level': 15,
   'app_lat': 37.541725158691406,
   'app_lng': 127.04335021972656,
   '법정동코드': '1120011400'},
  'zoom_level': {'google': 15, 'daum': 4},
  'zoom_level_v2': {'app': 5, 'web': 4}},
 37.541725158691406,
 127.04335021972656)

In [7]:
# 위도 경도 데이터를 가지고 geohash 데이터를 가져옴
# !pip install geohash2
import geohash2

In [33]:
# precision: 작을수록 큰범위, 클수록 작은 범위
geohash = geohash2.encode(lat, lng, precision=5)
geohash, lat, lng

('wydmd', 37.541725158691406, 127.04335021972656)

In [37]:
# geohash -> item ids
# url = 'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash=wydm6&rent_gteq=0&sales_type_in=%EC%A0%84%EC%84%B8%7C%EC%9B%94%EC%84%B8&service_type_eq=%EC%9B%90%EB%A3%B8'
url = 'https://apis.zigbang.com/property/apartments/location/v3?e=&geohash={}\
&markerType=large&n=&q=type=sales,price=0~-1,floorArea=0~-1&s=&serviceType[0]=apt&serviceType[1]=offer&w='.format(geohash)
response = requests.get(url)
datas = response.json()['filtered']
ids = [data['id'] for data in datas]
len(datas), len(ids), ids[:3]

(152, 152, [19722, 19990, 19929])

In [40]:
# ids -> items (매물정보)
ids_str = str(ids[:1]).replace(" ", "")

url = 'https://apis.zigbang.com/v2/danjis/{ids}'.format(ids=ids_str)
response = requests.get(url)
datas = response.json()
datas

{'name': 'BadRequestError',
 'code': 400,
 'message': [{'target': {'id': '[19722]',
    'accept-encoding': 'gzip',
    'host': 'apis-origin.zigbang.com',
    'user-agent': 'python-requests/2.24.0',
    'via': '1.1 c2fabe4c8485f75646a8199d0d11a3e5.cloudfront.net (CloudFront)',
    'x-amz-cf-id': '3s5-Xn27GzMLW2mug7a7OR9kbWjzI-w4xWquGUpdLG0LvIHbWYQ6jA==',
    'x-amzn-trace-id': 'Root=1-5f97b3ca-2c9328507711d5ba5e596c44',
    'x-forwarded-for': '106.242.79.230, 13.124.199.233',
    'x-forwarded-port': '443',
    'x-forwarded-proto': 'https',
    'x-apigateway-event': '%7B%22resource%22%3A%22%2F%7Bproxy%2B%7D%22%2C%22path%22%3A%22%2Fv2%2Fdanjis%2F%255B19722%255D%22%2C%22httpMethod%22%3A%22GET%22%2C%22headers%22%3A%7B%22Accept-Encoding%22%3A%22gzip%22%2C%22Host%22%3A%22apis-origin.zigbang.com%22%2C%22User-Agent%22%3A%22python-requests%2F2.24.0%22%2C%22Via%22%3A%221.1%20c2fabe4c8485f75646a8199d0d11a3e5.cloudfront.net%20(CloudFront)%22%2C%22X-Amz-Cf-Id%22%3A%223s5-Xn27GzMLW2mug7a7OR9kbWjzI-w4